**Chapter 3: Development Environment Setup**

Before we can containerize applications or orchestrate deployments, we must establish a robust, consistent development environment. This chapter provides platform-specific installation instructions and configuration best practices for the essential toolchain: Git for version control, Docker for containerization, Kubernetes for orchestration, and kubectl for cluster management. A properly configured local environment ensures that the code examples and exercises in subsequent chapters execute predictably across Linux, macOS, and Windows systems.

---

### 3.1 Local Development Tools

Modern cloud-native development requires more than just a text editor and compiler. The following tools form the minimum viable toolchain for professional CI/CD work. We recommend installing them in the order presented, as subsequent tools depend on earlier ones.

#### Essential Toolchain Overview

**Container Runtime:** Docker Desktop (macOS/Windows) or Docker Engine (Linux)
**Orchestration:** Minikube, Kind, or k3d for local Kubernetes clusters
**CLI Tools:** kubectl, Helm, Git
**Infrastructure:** Terraform (optional but recommended)
**Security:** cosign, Trivy (for image signing and scanning)

**System Requirements:**
- **RAM:** Minimum 8GB (16GB recommended for running multiple clusters)
- **CPU:** 4 cores minimum (8 recommended for parallel builds)
- **Disk:** 50GB free space (Docker images and Kubernetes volumes consume significant storage)
- **OS:** Linux (Ubuntu 22.04+), macOS (12+), or Windows 10/11 with WSL2

#### Shell Environment Configuration

Regardless of your operating system, configure a modern shell with productivity enhancements:

**Zsh with Oh-My-Zsh (Recommended for macOS/Linux):**
```bash
# Install Zsh
sudo apt install zsh  # Ubuntu/Debian
brew install zsh      # macOS with Homebrew

# Set as default
chsh -s $(which zsh)

# Install Oh-My-Zsh
sh -c "$(curl -fsSL https://raw.githubusercontent.com/ohmyzsh/ohmyzsh/master/tools/install.sh)"

# Enable useful plugins for Kubernetes/Docker
# Edit ~/.zshrc
plugins=(git docker kubectl helm)
```

**PowerShell (Windows):**
Install Windows Terminal and PowerShell 7+ for cross-platform compatibility:
```powershell
winget install Microsoft.WindowsTerminal
winget install Microsoft.PowerShell
```

---

### 3.2 Installing Docker

Docker is the industry-standard container runtime. Installation methods vary significantly by operating system, and choosing the correct approach is critical for Kubernetes integration.

#### Linux (Ubuntu/Debian) - Docker Engine

On Linux, install Docker Engine directly rather than Docker Desktop for better performance and Linux-native workflow.

**Step 1: Remove old versions**
```bash
sudo apt-get remove docker docker-engine docker.io containerd runc
```

**Step 2: Install prerequisites**
```bash
sudo apt-get update
sudo apt-get install ca-certificates curl gnupg lsb-release
```

**Step 3: Add Docker's official GPG key**
```bash
sudo install -m 0755 -d /etc/apt/keyrings
curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo gpg --dearmor -o /etc/apt/keyrings/docker.gpg
sudo chmod a+r /etc/apt/keyrings/docker.gpg
```

**Step 4: Set up the repository**
```bash
echo \
  "deb [arch="$(dpkg --print-architecture)" signed-by=/etc/apt/keyrings/docker.gpg] https://download.docker.com/linux/ubuntu \
  "$(. /etc/os-release && echo "$VERSION_CODENAME")" stable" | \
  sudo tee /etc/apt/sources.list.d/docker.list > /dev/null
```

**Step 5: Install Docker Engine**
```bash
sudo apt-get update
sudo apt-get install docker-ce docker-ce-cli containerd.io docker-buildx-plugin docker-compose-plugin
```

**Step 6: Manage Docker as non-root user**
```bash
# Create docker group
sudo groupadd docker

# Add your user to docker group
sudo usermod -aG docker $USER

# Apply group membership (log out and back in, or use):
newgrp docker

# Verify
docker run hello-world
```

**Important:** Running Docker as root is a security risk. Always use the docker group approach.

#### macOS - Docker Desktop

**Intel Macs:**
Download Docker Desktop for Mac (Intel) from [docker.com](https://www.docker.com/products/docker-desktop/).

**Apple Silicon (M1/M2/M3):**
Download Docker Desktop for Mac (Apple Silicon). While Docker supports x86_64 emulation, build multi-platform images to ensure production compatibility:

```bash
# Verify architecture support
docker run --platform linux/amd64 hello-world
docker run --platform linux/arm64 hello-world
```

**Configuration Best Practices:**
1. Open Docker Desktop → Settings → Resources
2. Allocate at least 4GB RAM (8GB recommended)
3. Enable VirtioFS for faster file sharing (macOS 12.5+)
4. Enable "Allow default Docker socket to be used" (required for some Kubernetes tools)

#### Windows - Docker Desktop with WSL2

Windows requires Windows Subsystem for Linux 2 (WSL2) for optimal Docker performance.

**Step 1: Install WSL2**
```powershell
wsl --install -d Ubuntu-22.04
```

**Step 2: Install Docker Desktop**
Download from [docker.com](https://www.docker.com/products/docker-desktop/) and install.

**Step 3: Enable WSL2 Backend**
Open Docker Desktop → Settings → General:
- Check "Use the WSL 2 based engine"
- Check "Add the *.docker.internal names to the host's /etc/hosts file"

**Step 4: Enable Kubernetes (Optional but recommended)**
Docker Desktop includes a single-node Kubernetes cluster:
Settings → Kubernetes → Check "Enable Kubernetes"

**Step 5: Integrate with WSL distro**
Settings → Resources → WSL Integration → Enable integration with your default WSL distro.

**Verification:**
```bash
# In WSL2 terminal
docker --version
docker-compose --version
```

#### Docker Post-Installation Configuration

**Configure Docker Daemon (Linux):**
Create `/etc/docker/daemon.json` for production-parity settings:

```json
{
  "exec-opts": ["native.cgroupdriver=systemd"],
  "log-driver": "json-file",
  "log-opts": {
    "max-size": "100m",
    "max-file": "3"
  },
  "storage-driver": "overlay2",
  "features": {
    "buildkit": true
  }
}
```

Restart Docker:
```bash
sudo systemctl restart docker
```

**Enable BuildKit (All Platforms):**
BuildKit is the modern builder with improved performance and features:
```bash
# Add to ~/.bashrc or ~/.zshrc
export DOCKER_BUILDKIT=1
```

---

### 3.3 Installing Kubernetes (Minikube/Kind)

While cloud providers offer managed Kubernetes, local development requires a lightweight cluster. We present two options: **Minikube** (VM-based, closer to production) and **Kind** (Docker-based, faster, ideal for CI testing).

#### Option A: Minikube (Recommended for Beginners)

Minikube creates a single-node Kubernetes cluster in a VM (or container), providing the closest experience to production clusters.

**Installation:**

**Linux:**
```bash
curl -LO https://storage.googleapis.com/minikube/releases/latest/minikube-linux-amd64
sudo install minikube-linux-amd64 /usr/local/bin/minikube
```

**macOS:**
```bash
brew install minikube
```

**Windows:**
```powershell
choco install minikube
# or
winget install Kubernetes.minikube
```

**Starting Minikube:**
```bash
# Start with sufficient resources for CI/CD workloads
minikube start --driver=docker --cpus=4 --memory=8192 --disk-size=20g

# Verify
minikube status

# Enable useful addons
minikube addons enable ingress
minikube addons enable metrics-server
minikube addons enable dashboard
```

**Driver Selection:**
- **Docker:** Fastest, runs K8s inside containers (recommended)
- **VirtualBox:** True VM isolation (useful for testing network policies)
- **KVM2:** Linux native virtualization (best performance on Linux)
- **HyperKit:** macOS native (deprecated in favor of Docker driver)

**Accessing Minikube Services:**
Unlike Kind, Minikube provides a stable IP:
```bash
minikube ip  # Returns cluster IP (e.g., 192.168.49.2)
```

#### Option B: Kind (Kubernetes in Docker)

Kind (Kubernetes IN Docker) runs cluster nodes as Docker containers. It is faster to start, uses fewer resources, and is ideal for testing multi-node scenarios.

**Installation:**
```bash
# Linux/macOS
curl -Lo ./kind https://kind.sigs.k8s.io/dl/v0.20.0/kind-linux-amd64
chmod +x ./kind
sudo mv ./kind /usr/local/bin/kind

# macOS with Homebrew
brew install kind

# Windows
choco install kind
```

**Creating a Cluster:**
Create a configuration file for a multi-node cluster (simulating production):

```yaml
# kind-config.yaml
kind: Cluster
apiVersion: kind.x-k8s.io/v1alpha4
nodes:
  - role: control-plane
    kubeadmConfigPatches:
      - |
        kind: InitConfiguration
        nodeRegistration:
          kubeletExtraArgs:
            node-labels: "ingress-ready=true"
    extraPortMappings:
      - containerPort: 80
        hostPort: 80
        protocol: TCP
      - containerPort: 443
        hostPort: 443
        protocol: TCP
  - role: worker
  - role: worker
```

Start the cluster:
```bash
kind create cluster --name cicd-cluster --config kind-config.yaml
```

**Loading Local Images:**
A key advantage of Kind is loading local Docker images without pushing to a registry:
```bash
# Build image locally
docker build -t myapp:v1 .

# Load into Kind cluster
kind load docker-image myapp:v1 --name cicd-cluster

# Use in deployment
kubectl create deployment myapp --image=myapp:v1
```

#### Option C: k3d (Lightweight k3s)

k3d runs k3s (lightweight Kubernetes) in Docker. It is extremely fast and resource-efficient, perfect for local CI/CD testing.

**Installation:**
```bash
# Linux/macOS
wget -q -O - https://raw.githubusercontent.com/k3d-io/k3d/main/install.sh | bash

# macOS
brew install k3d
```

**Create Cluster:**
```bash
k3d cluster create cicd-cluster --servers 1 --agents 2 --port "8080:80@loadbalancer"
```

**Comparison Matrix:**

| Feature | Minikube | Kind | k3d |
|---------|----------|------|-----|
| **Startup Time** | ~2 minutes | ~30 seconds | ~15 seconds |
| **Resource Usage** | High | Medium | Low |
| **Multi-Node Support** | Limited | Excellent | Excellent |
| **Production Similarity** | High | Medium | Medium |
| **Best For** | Learning, local dev | CI testing, multi-node | Resource-constrained systems |

**Recommendation:** Use **Minikube** if you are new to Kubernetes and want a production-like experience. Use **Kind** if you need to test multi-node scenarios or have limited resources.

---

### 3.4 Installing kubectl

**kubectl** is the command-line interface for interacting with Kubernetes clusters. It is the primary tool you will use for deployment, debugging, and cluster management throughout your CI/CD journey.

#### Installation

**Linux:**
```bash
curl -LO "https://dl.k8s.io/release/$(curl -L -s https://dl.k8s.io/release/stable.txt)/bin/linux/amd64/kubectl"
sudo install -o root -g root -m 0755 kubectl /usr/local/bin/kubectl
```

**macOS:**
```bash
brew install kubectl
# or
curl -LO "https://dl.k8s.io/release/$(curl -L -s https://dl.k8s.io/release/stable.txt)/bin/darwin/amd64/kubectl"
chmod +x ./kubectl
sudo mv ./kubectl /usr/local/bin/kubectl
```

**Windows:**
```powershell
choco install kubernetes-cli
# or
winget install Kubernetes.kubectl
```

#### Configuration and Contexts

kubectl uses configuration files (kubeconfig) to manage cluster connections. The default location is `~/.kube/config`.

**View current configuration:**
```bash
kubectl config view
```

**Managing Multiple Contexts:**
As you work with local Minikube, staging, and production clusters, context switching becomes essential:

```bash
# View current contexts
kubectl config get-contexts

# Set context to minikube
kubectl config use-context minikube

# Set context to production (be careful!)
kubectl config use-context production-cluster

# View current context
kubectl config current-context
```

**Namespace Scoping:**
Always work in specific namespaces rather than default:

```bash
# Set namespace for current context
kubectl config set-context --current --namespace=development

# Verify
kubectl config view --minify | grep namespace:
```

#### Shell Autocompletion

Enable tab completion for faster command entry:

**Bash:**
```bash
echo 'source <(kubectl completion bash)' >> ~/.bashrc
```

**Zsh:**
```bash
echo 'source <(kubectl completion zsh)' >> ~/.zshrc
```

#### Useful Aliases

Add these to your shell profile for productivity:

```bash
alias k='kubectl'
alias kgp='kubectl get pods'
alias kgd='kubectl get deployments'
alias kgs='kubectl get services'
alias kaf='kubectl apply -f'
alias kdf='kubectl delete -f'
alias kl='kubectl logs'
alias klf='kubectl logs -f'  # Follow logs
alias kex='kubectl exec -it'  # Interactive exec
alias kctx='kubectl config use-context'
alias kns='kubectl config set-context --current --namespace'
```

#### Verification

Ensure kubectl communicates with your cluster:
```bash
kubectl cluster-info
# Expected output: Kubernetes control plane is running at https://...

kubectl get nodes
# Expected output: NAME       STATUS   ROLES           AGE   VERSION
#                  minikube   Ready    control-plane   10m   v1.28.3
```

---

### 3.5 Installing Git

Git is the foundation of CI/CD. While you likely have Git installed, configuring it correctly for automation and security is crucial.

#### Installation

**Linux:**
```bash
sudo apt-get update
sudo apt-get install git
```

**macOS:**
```bash
# Usually pre-installed, but update via:
brew install git
```

**Windows:**
Install Git for Windows (includes Git Bash):
```powershell
winget install Git.Git
```

#### Essential Configuration

Configure identity (required for commits):
```bash
git config --global user.name "Your Name"
git config --global user.email "your.email@company.com"
```

Set default branch name:
```bash
git config --global init.defaultBranch main
```

Configure line endings (critical for cross-platform teams):
```bash
# macOS/Linux
git config --global core.autocrlf input

# Windows
git config --global core.autocrlf true
```

Set default editor (for interactive rebase):
```bash
git config --global core.editor "code --wait"  # VS Code
# or
git config --global core.editor "vim"
```

#### Security Configuration

**Credential Helper:**
Store credentials securely:
```bash
# macOS (Keychain)
git config --global credential.helper osxkeychain

# Linux (Libsecret)
git config --global credential.helper /usr/share/doc/git/contrib/credential/libsecret/git-credential-libsecret

# Windows (Credential Manager)
git config --global credential.helper manager
```

**GPG Signing (Recommended):**
Sign commits cryptographically to verify identity:

```bash
# Generate GPG key
gpg --full-generate-key

# List keys
gpg --list-secret-keys --keyid-format=long

# Configure Git to use the key
git config --global user.signingkey YOUR_KEY_ID
git config --global commit.gpgsign true
```

#### Git Hooks Setup

Install pre-commit framework for automated checks:
```bash
pip install pre-commit
# or
brew install pre-commit
```

Create `.pre-commit-config.yaml` in your home directory for global hooks, or per repository:
```yaml
repos:
  - repo: https://github.com/pre-commit/pre-commit-hooks
    rev: v4.4.0
    hooks:
      - id: trailing-whitespace
      - id: check-yaml
      - id: check-added-large-files
        args: ['--maxkb=1000']
      - id: detect-private-key
```

---

### 3.6 IDE and Editor Configurations

Your IDE is your primary interface with code, containers, and Kubernetes. Proper configuration enables seamless navigation between local code, containerized applications, and cluster resources.

#### Visual Studio Code (Recommended)

**Extensions for CI/CD Development:**

1. **Docker** (Microsoft): Dockerfile syntax highlighting, image management, container debugging
2. **Kubernetes** (Microsoft): Manifest editing, cluster navigation, Helm support
3. **YAML** (Red Hat): Schema validation for Kubernetes manifests
4. **GitLens**: Enhanced Git visualization, blame annotations
5. **Remote - Containers**: Develop inside containers for environment parity
6. **Terraform**: Infrastructure as code support
7. **ShellCheck**: Bash script linting

**Settings Configuration:**
Create `.vscode/settings.json` in your workspace:
```json
{
  "editor.formatOnSave": true,
  "editor.insertSpaces": true,
  "editor.tabSize": 2,
  "yaml.schemas": {
    "kubernetes": ["*.yaml", "*.yml"]
  },
  "docker.enableLinting": true,
  "terminal.integrated.defaultProfile.linux": "zsh",
  "terminal.integrated.defaultProfile.osx": "zsh",
  "files.associations": {
    "*.dockerfile": "dockerfile"
  }
}
```

**Launch Configuration for Debugging:**
Create `.vscode/launch.json` for container debugging:
```json
{
  "version": "0.2.0",
  "configurations": [
    {
      "name": "Docker: Attach to Node",
      "type": "node",
      "request": "attach",
      "port": 9229,
      "address": "localhost",
      "localRoot": "${workspaceFolder}",
      "remoteRoot": "/app"
    }
  ]
}
```

#### JetBrains IntelliJ IDEA / GoLand / PyCharm

**Plugins:**
- Kubernetes: Manifest editing, cluster view, Helm support
- .env files support: Environment variable management
- Docker: Container management, image building

**Configuration:**
Enable Docker integration:
Settings → Build, Execution, Deployment → Docker → Connect to Docker daemon

#### Vim/Neovim

For terminal-centric workflows:
```vim
" Plugins (using vim-plug)
Plug 'fatih/vim-go'
Plug 'stephpy/vim-yaml'
Plug 'hashivim/vim-terraform'
Plug 'ekalinin/Dockerfile.vim'
```

---

### 3.7 Setting Up a Test Repository

With tools installed, we create a structured repository that will serve as our learning environment for subsequent chapters. This repository follows industry standards for monorepo structure, ready for CI/CD implementation.

#### Repository Initialization

```bash
# Create directory
mkdir ~/dev-handbook-cicd
cd ~/dev-handbook-cicd

# Initialize Git
git init
git checkout -b main

# Create standard directory structure
mkdir -p src/api src/web infra/k8s infra/terraform ci/scripts docs
mkdir -p .github/workflows  # For GitHub Actions later

# Create initial files
touch README.md
touch .gitignore
touch .dockerignore
touch Makefile
```

#### Gitignore Configuration

Create a comprehensive `.gitignore`:
```gitignore
# OS files
.DS_Store
Thumbs.db

# IDE
.idea/
.vscode/
*.swp
*.swo

# Dependencies
node_modules/
vendor/
__pycache__/
*.pyc

# Build artifacts
dist/
build/
target/
*.exe
*.dll
*.so

# Environment variables
.env
.env.local
.env.*.local
*.pem
*.key

# Kubernetes
kubeconfig
*.kubeconfig

# Terraform
.terraform/
*.tfstate
*.tfstate.*
crash.log

# Docker
.docker/
```

#### Makefile for Common Tasks

Create a `Makefile` to standardize commands across team members:

```makefile
.PHONY: help build test clean k8s-up k8s-down

help: ## Display this help
	@awk 'BEGIN {FS = ":.*##"; printf "\nUsage:\n  make \033[36m<target>\033[0m\n"} /^[a-zA-Z_-]+:.*?##/ { printf "  \033[36m%-15s\033[0m %s\n", $$1, $$2 }' $(MAKEFILE_LIST)

build: ## Build Docker images
	docker build -t dev-handbook/api:latest -f src/api/Dockerfile src/api
	docker build -t dev-handbook/web:latest -f src/web/Dockerfile src/web

test: ## Run tests
	cd src/api && npm test
	cd src/web && npm test

k8s-up: ## Start local Kubernetes cluster
	minikube start --driver=docker --cpus=4 --memory=8192
	kubectl apply -f infra/k8s/namespace.yaml

k8s-down: ## Stop local Kubernetes cluster
	minikube stop

clean: ## Clean up Docker images and volumes
	docker system prune -f
	docker volume prune -f
```

#### Initial Commit

```bash
git add .
git commit -m "chore: initial repository structure

- Add directory structure for monorepo
- Configure .gitignore for Node.js, Kubernetes, Terraform
- Add Makefile for standard commands"
```

---

### 3.8 Verifying Your Setup

Before proceeding to containerization concepts, verify that all components function correctly and integrate properly.

#### Verification Checklist Script

Create `scripts/verify-setup.sh`:

```bash
#!/bin/bash
set -e

echo "🔍 Verifying CI/CD Development Environment..."

# Check Git
echo "✓ Git version: $(git --version)"

# Check Docker
if docker --version > /dev/null 2>&1; then
    echo "✓ Docker version: $(docker --version)"
    docker run --rm hello-world > /dev/null 2>&1 && echo "✓ Docker daemon running"
else
    echo "✗ Docker not installed or not running"
    exit 1
fi

# Check Kubernetes
if kubectl version --client > /dev/null 2>&1; then
    echo "✓ kubectl version: $(kubectl version --client -o json | grep gitVersion | head -1)"
else
    echo "✗ kubectl not installed"
    exit 1
fi

# Check cluster connectivity
if kubectl cluster-info > /dev/null 2>&1; then
    echo "✓ Kubernetes cluster connected"
    echo "  Nodes: $(kubectl get nodes -o name | wc -l)"
else
    echo "⚠ No Kubernetes cluster running (run 'minikube start' or 'kind create cluster')"
fi

# Check resources
echo ""
echo "📊 Resource Allocation:"
echo "  Docker CPUs: $(docker system info 2>/dev/null | grep CPUs | awk '{print $2}')"
echo "  Docker Memory: $(docker system info 2>/dev/null | grep "Total Memory" | awk '{print $3 $4}')"

echo ""
echo "🎉 Environment verification complete!"
```

Make it executable and run:
```bash
chmod +x scripts/verify-setup.sh
./scripts/verify-setup.sh
```

#### Test Deployment

Deploy a simple Nginx pod to verify the entire toolchain:

```bash
# Create test deployment
kubectl create deployment nginx-test --image=nginx:alpine --port=80

# Expose it
kubectl expose deployment nginx-test --type=NodePort --port=80

# Check status
kubectl get pods -l app=nginx-test
kubectl get svc nginx-test

# Access it (Minikube)
minikube service nginx-test

# Or port-forward (works with any cluster)
kubectl port-forward svc/nginx-test 8080:80
# Open browser to http://localhost:8080

# Clean up
kubectl delete deployment nginx-test
kubectl delete service nginx-test
```

#### Troubleshooting Common Issues

**Issue:** `docker: Got permission denied`
**Solution:** Ensure your user is in the docker group and you have run `newgrp docker` or logged out/in.

**Issue:** `kubectl: Unable to connect to the server`
**Solution:** Verify Minikube/Kind is running:
```bash
minikube status
# or
kind get clusters
```

**Issue:** Minikube stuck on "Starting control plane"
**Solution:** Check virtualization support:
```bash
# Linux
sudo apt install cpu-checker && kvm-ok

# Or use Docker driver instead of VM
minikube delete
minikube start --driver=docker
```

**Issue:** Port 80 already in use (macOS/Windows)
**Solution:** Use higher port numbers or stop conflicting services (Apache, Nginx, IIS).

---

### Chapter Summary and Preview

In this chapter, you established a professional-grade development environment equipped for cloud-native CI/CD work. You installed and configured **Docker** with proper security settings and BuildKit enabled, selected and deployed a local **Kubernetes** cluster using Minikube or Kind, configured **kubectl** with contexts and aliases for efficient cluster management, set up **Git** with security features like GPG signing and pre-commit hooks, configured your **IDE** with extensions for container and Kubernetes development, created a **test repository** with industry-standard structure, and verified the entire toolchain integration.

Your environment now mirrors the production-grade setups used by platform engineering teams. The Docker daemon runs with appropriate resource allocation, Kubernetes is ready to orchestrate containers, and your repository is structured to support the artifacts and configurations we will create in subsequent chapters.

In **Chapter 4: Docker Basics**, we will put this environment to immediate use. You will learn the fundamental concepts of containerization—what containers are, how they differ from virtual machines, and the architecture of the Docker platform. We will run your first containers, understand the container lifecycle, and explore the Docker image system. This hands-on chapter transitions from environment setup to practical containerization, teaching you to package applications in a way that ensures consistency from development through production. With your newly configured environment, you will execute every command and witness firsthand how Docker solves the "it works on my machine" problem that has plagued software development for decades.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='../11. real_world_projects/2. microservices_architecture.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='../11. real_world_projects/3. multi_environment_enterprise_app.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
